<a href="https://colab.research.google.com/github/jonathan-krantz/chalmers-advanced-python/blob/main/Assignment5_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Edvin Angel (4 hrs), Noah Johnsson (4hrs), Jonathan Krantz (4hrs). Group 40


# DAT405 Introduction to Data Science and AI 
## 2022-2023, Reading Period 3
## Assignment 5: Reinforcement learning and classification

The exercise takes place in a notebook environment where you can chose to use Jupyter or Google Colabs. We recommend you use Google Colabs as it will facilitate remote group-work and makes the assignment less technical. 
Hints:
You can execute certain linux shell commands by prefixing the command with `!`. You can insert Markdown cells and code cells. The first you can use for documenting and explaining your results the second you can use writing code snippets that execute the tasks required.  

This assignment is about **sequential decision making** under uncertainty (Reinforcement learning). In a sequential decision process, the process jumps between different states (the environment), and in each state the decision maker, or agent, chooses among a set of actions. Given the state and the chosen action, the process jumps to a new state. At each jump the decision maker receives a reward, and the objective is to find a sequence of decisions (or an optimal policy) that maximizes the accumulated rewards.

We will use **Markov decision processes** (MDPs) to model the environment, and below is a primer on the relevant background theory. 



* To make things concrete, we will first focus on decision making under **no** uncertainity (question 1 and 2), i.e, given we have a world model, we can calculate the exact and optimal actions to take in it. We will first introduce **Markov Decision Process (MDP)** as the world model. Then we give one algorithm (out of many) to solve it.

* (Optional) Next we will work through one type of reinforcement learning algorithm called Q-learning (question 3). Q-learning is an algorithm for making decisions under uncertainity, where uncertainity is over the possible world model (here MDP). It will find the optimal policy for the **unknown** MDP, assuming we do infinite exploration.

* Finally, in question 4 you will be asked to explain differences between reinforcement learning and supervised learning and in question 5 write about decision trees and random forests.

## Primer
### Decision Making
The problem of **decision making under uncertainty** (commonly known as **reinforcement learning**) can be broken down into
two parts. First, how do we learn about the world? This involves both the
problem of modeling our initial uncertainty about the world, and that of drawing conclusions from evidence and our initial belief. Secondly, given what we
currently know about the world, how should we decide what to do, taking into
account future events and observations that may change our conclusions?
Typically, this will involve creating long-term plans covering possible future
eventualities. That is, when planning under uncertainty, we also need to take
into account what possible future knowledge could be generated when implementing our plans. Intuitively, executing plans which involve trying out new
things should give more information, but it is hard to tell whether this information will be beneficial. The choice between doing something which is already
known to produce good results and experiment with something new is known
as the **exploration-exploitation dilemma**.

### The exploration-exploitation trade-off

Consider the problem of selecting a restaurant to go to during a vacation. Lets say the
best restaurant you have found so far was **Les Epinards**. The food there is
usually to your taste and satisfactory. However, a well-known recommendations
website suggests that **King’s Arm** is really good! It is tempting to try it out. But
there is a risk involved. It may turn out to be much worse than **Les Epinards**,
in which case you will regret going there. On the other hand, it could also be
much better. What should you do?
It all depends on how much information you have about either restaurant,
and how many more days you’ll stay in town. If this is your last day, then it’s
probably a better idea to go to **Les Epinards**, unless you are expecting **King’s
Arm** to be significantly better. However, if you are going to stay there longer,
trying out **King’s Arm** is a good bet. If you are lucky, you will be getting much
better food for the remaining time, while otherwise you will have missed only
one good meal out of many, making the potential risk quite small.

### Markov Decision Processes
Markov Decision Processes (MDPs) provide a mathematical framework for modeling sequential decision making under uncertainty. An *agent* moves between *states* in a *state space* choosing *actions* that affects the transition probabilities between states, and the subsequent *rewards* recieved after a jump. This is then repeated a finite or infinite number of epochs. The objective, or the *solution* of the MDP, is to optimize the accumulated rewards of the process.

Thus, an MDP consists of five parts: 

* Decision epochs: $t={1,2,...,T}$, where $T\leq \infty$
* State space: $S=\{s_1,s_2,...,s_N\}$ of the underlying environment
* Action space $A=\{a_1,a_2,...,a_K\}$ available to the decision maker at each decision epoch
* Transition probabilities $p(s_{t+1}|s_t,a_t)$ for jumping from state $s_t$ to state $s_{t+1}$ after taking action $a_t$
* Reward functions $R_t = r(a_t,s_t,s_{t+1})$ resulting from the chosen action and subsequent transition

A *decision policy* is a function $\pi: s \rightarrow a$, that gives instructions on what action to choose in each state. A policy can either be *deterministic*, meaning that the action is given for each state, or *randomized* meaning that there is a probability distribution over the set of possible actions for each state. Given a specific policy $\pi$ we can then compute the the *expected total reward* when starting in a given state $s_1 \in S$, which is also known as the *value* for that state, 

$$V^\pi (s_1) = E\left[ \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) {\Large |} s_1\right] = \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) p(s_{t+1} | a_t,s_t)$$ 

where $a_t = \pi(s_t)$. To ensure convergence and to control how much credit to give to future rewards, it is common to introduce a *discount factor* $\gamma \in [0,1]$. For instance, if we think all future rewards should count equally, we would use $\gamma = 1$, while if we value near-future rewards higher than more distant rewards, we would use $\gamma < 1$. The expected total *discounted* reward then becomes

$$V^\pi( s_1) = \sum_{t=1}^T \gamma^{t-1} r(s_t,a_t, s_{t+1}) p(s_{t+1} | s_t, a_t) $$

Now, to find the *optimal* policy we want to find the policy $\pi^*$ that gives the highest total reward $V^*(s)$ for all $s\in S$. That is, we want to find the policy where

$$V^*(s) \geq V^\pi(s), s\in S$$

To solve this we use a dynamic programming equation called the *Bellman equation*, given by

$$V(s) = \max_{a\in A} \left\{\sum_{s'\in S} p(s'|s,a)( r(s,a,s') +\gamma V(s')) \right\}$$

It can be shown that if $\pi$ is a policy such that $V^\pi$ fulfills the Bellman equation, then $\pi$ is an optimal policy.

A real world example would be an inventory control system. The states could be the amount of items we have in stock, and the actions would be the amount of items to order at the end of each month. The discrete time would be each month and the reward would be the profit. 


## Question 1

The first question covers a deterministic MPD, where the action is directly given by the state, described as follows:

* The agent starts in state **S** (see table below)
* The actions possible are **N** (north), **S** (south), **E** (east), and **W** west. 
* The transition probabilities in each box are deterministic (for example P(s'|s,N)=1 if s' north of s). Note, however, that you cannot move outside the grid, thus all actions are not available in every box.
* When reaching **F**, the game ends (absorbing state).
* The numbers in the boxes represent the rewards you receive when moving into that box. 
* Assume no discount in this model: $\gamma = 1$
    
    
| | | |
|----------|----------|---------|
|-1 |1|**F**|
|0|-1|1|  
|-1 |0|-1|  
|**S**|-1|1|

Let $(x,y)$ denote the position in the grid, such that $S=(0,0)$ and $F=(2,3)$.

**1a)** What is the optimal path of the MDP above? Is it unique? Submit the path as a single string of directions. E.g. NESW will make a circle.

**1b)** What is the optimal policy (i.e. the optimal action in each state)? It is helpful if you draw the arrows/letters in the grid.

**1c)** What is expected total reward for the policy in 1a)?


1a) First and foremost, we make sure that there is no infinite loop. A circle of NESW where the sum of all numbers included in the circle is positive. This would make this problem impossible to answer since the program would circle around the loop in infinity to successively increase the reward. 

We have no infinite loop and the optimal path for the MDP is therefore EENNN (alternatively EENNWNE which also results in the reward 0). 

1b)

Determining policy involves finding the optimal decision-making strategy for an agent that maximize its culmative rewards. The policy specifies which action the agent should take in each state. The optimal policy in each state is as follows:



*   State (0,0) -> N/E is -1
*   State (0,1) -> N/E is 0 
*   State (0,2) -> N/S/E is -1
*   State (0,3) -> E is 1
*   State (1,0) -> E is 1
*   State (1,1) -> N/S/E/W is -1
*   State (1,2) -> N/E is 1
*   State (1,3) -> E to absorbing state
*   State (2,0) -> N/W is -1
*   State (2,1) -> N/S is 1
*   State (2,2) -> N to absorbing state.

Optimal policy illustrated as a grid:

| | | |
|----------|----------|---------|
|→|→|**F**|
|↑↓→|↑→|↑|  
|↑→|↑↓→←|↑↓|  
|**S**↑→|→|←↑|

1c) The expected value is 0.

## Value Iteration

For larger problems we need to utilize algorithms to determine the optimal policy $\pi^*$. *Value iteration* is one such algorithm that iteratively computes the value for each state. Recall that for a policy to be optimal, it must satisfy the Bellman equation above, meaning that plugging in a given candidate $V^*$ in the right-hand side (RHS) of the Bellman equation should result in the same $V^*$ on the left-hand side (LHS). This property will form the basis of our algorithm. Essentially, it can be shown that repeated application of the RHS to any intial value function $V^0(s)$ will eventually lead to the value $V$ which statifies the Bellman equation. Hence repeated application of the Bellman equation will also lead to the optimal value function. We can then extract the optimal policy by simply noting what actions that satisfy the equation.    

The process of repeated application of the Bellman equation is what we here call the _value iteration_ algorithm. It practically procedes as follows:

```
epsilon is a small value, threshold
for x from i to infinity 
do
    for each state s
    do
        V_k[s] = max_a Σ_s' p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
    end
    if  |V_k[s]-V_k-1[s]| < epsilon for all s
        for each state s,
        do
            π(s)=argmax_a ∑_s′ p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
            return π, V_k 
        end
end

```

**Example:** We will illustrate the value iteration algorithm by going through two iterations. Below is a 3x3 grid with the rewards given in each state. Assume now that given a certain state $s$ and action $a$, there is a probability 0.8 that that action will be performed and a probability 0.2 that no action is taken. For instance, if we take action **E** in state $(x,y)$ we will go to $(x+1,y)$ 80 percent of the time (given that that action is available in that state), and remain still 20 percent of the time. We will use have a discount factor $\gamma = 0.9$. Let the initial value be $V^0(s)=0$ for all states $s\in S$. 

**Reward**:

| | | |  
|----------|----------|---------|  
|0|0|0|
|0|10|0|  
|0|0|0|  


**Iteration 1**: The first iteration is trivial, $V^1(s)$ becomes the $\max_a \sum_{s'} p(s'|s,a) r(s,a,s')$ since $V^0$ was zero for all $s'$. The updated values for each state become

| | | |  
|----------|----------|---------|  
|0|8|0|
|8|2|8|  
|0|8|0|  
  
**Iteration 2**:  
  
Staring with cell (0,0) (lower left corner): We find the expected value of each move:  
Action **S**: 0  
Action **E**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **N**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **W**: 0

Hence any action between **E** and **N** would be best at this stage.

Similarly for cell (1,0):

Action **N**: 0.8( 10 + 0.9 \* 2) + 0.2(0 + 0.9 \* 8) = 10.88 (Action **N** is the maximizing action)  

Similar calculations for remaining cells give us:

| | | |  
|----------|----------|---------|  
|5.76|10.88|5.76|
|10.88|8.12|10.88|  
|5.76|10.88|5.76|  


## Question 2

**2a)** Code the value iteration algorithm just described here, and show the converging optimal value function and the optimal policy for the above 3x3 grid.

**2b)** Explain why the result of 2a) does not depend on the initial value $V_0$.

**2c)** Describe your interpretation of the discount factor $\gamma$. What would happen in the two extreme cases $\gamma = 0$ and $\gamma = 1$? Given some MDP, what would be important things to consider when deciding on which value of $\gamma$ to use?

**2a)**

In [12]:
import copy
import numpy as np

def get_matrix_by_iteration(discount):

  # Choosing epsilon close to zero
  e = 0.001

  # Random start value
  d = 1

  # Creating reward matrix
  rewards = np.asarray([[0, 0, 0], [0, 10, 0], [0, 0, 0]], dtype=float)

  # Creating epsilon comparison matrix
  comparison_matrix = np.zeros((3, 3), dtype=float)

  # Creating the environment
  environment_matrix = np.zeros((3, 3), dtype=float)


  rows, columns = environment_matrix.shape

  while d > e:

      for y in range(rows):
          for x in range(columns):

              N = (x, y + 1)
              S = (x, y - 1)
              E = (x + 1, y)
              W = (x - 1, y)

              actions = [N, E, W, S]

              # Expected reward per action
              reward = 0.0

              # Highest expected reward per action
              max_reward = 0.0

              # Trying each action in actions, catching with IndexError if the 
              # agent choose an action so it goes out of bounds..
              for action in actions:
                  try:
                      reward = 0.8*(rewards[action] + environment_matrix[action] * discount) + \
                              0.2*(rewards[x, y] + environment_matrix[x, y] * discount)
                      
                      # Compare the current reward with maximum reward. 
                      # If current reward is higher max reward is updated. 
                      max_reward = max(max_reward, reward)
                  except IndexError:
                      pass

                  # Environment update
                  environment_matrix[x, y] = max_reward

      # Compare the difference between the current environment and previous environment.
      d = np.abs(environment_matrix - comparison_matrix).sum()

      # Make a copy of the current environment and assigns it to the comparison matrix, 
      # which will use to check the difference between the current and previous environment.
      comparison_matrix = copy.deepcopy(environment_matrix)

  # Prints the results converging value
  for x in range(len(environment_matrix)):
      print(np.around(environment_matrix[x], 2))

get_matrix_by_iteration(discount=0.9)

[44.15 48.43 41.3 ]
[50.61 46.53 47.81]
[44.14 48.39 41.04]


**2b)**
The result of 2A) does not depend on the initial value V0 because the algorithm 
is an iterative process that updates the values of each state based on the 
values of its neighboring states. The updates are made according to the 
Bellman equation, which takes into account the expected reward and the discount 
factor. 

The algorithm starts with an initial value for each state, but as the 
iterations progress, the values are updated and converge to the optimal values. 

The convergence occurs because the updates are based on the current 
values of the neighboring states, and these values also converge to their 
optimal values. Therefore, the final result does not depend on the initial 
values, but rather on the rewards and discount factor of the environment. 
Changing the initial values may affect the speed of convergence, but it 
will not affect the final result.

**2c)**

In [13]:
# discount = 0
get_matrix_by_iteration(discount=0)

[0. 8. 0.]
[8. 2. 8.]
[0. 8. 0.]


In [14]:
# discount = 1
get_matrix_by_iteration(discount=1)

[64435.27 64413.08 63433.1 ]
[64439.71 64429.71 63674.08]
[64422.43 64336.   61656.65]


When calling the function using discount = 0, the agent will only care about immediate rewards, not taking into account future rewards. The optimal policy will therefore move to the cell with a reward of 10 and then stay there indefinetly. As a result, the center cell will have a value of 2 and the surrounding cells have a value of 8.

Aternatively, calling the function using discount = 1 will cause the agent to care equally about immediate and future rewards. This entails that the optimal policy will be to explore the envirnmont and collect as much reward as possible, which would explain the large values seen in the output matrix. 

## Reinforcement Learning (RL) (Theory for optional question 3)
Until now, we understood that knowing the MDP, specifically $p(s'|a,s)$ and $r(s,a,s')$ allows us to efficiently find the optimal policy using the value iteration algorithm. Reinforcement learning (RL) or decision making under uncertainity, however, arises from the question of making optimal decisions without knowing the true world model (the MDP in this case).

So far we have defined the value function for a policy through $V^\pi$. Let's now define the *action-value function*

$$Q^\pi(s,a) = \sum_{s'} p(s'|a,s) [r(s,a,s') + \gamma V^\pi(s')]$$

The value function and the action-value function are directly related through

$$V^\pi (s) = \max_a Q^\pi (s,a)$$

i.e, the value of taking action $a$ in state $s$ and then following the policy $\pi$ onwards. Similarly to the value function, the optimal $Q$-value equation is:

$$Q^*(s,a) = \sum_{s'} p(s'|a,s) [r(s,a,s') + \gamma V^*(s')]$$

and the relationship between $Q^*(s,a)$ and $V^*(s)$ is simply

$$V^*(s) = \max_{a\in A} Q^*(s,a).$$

#### Q-learning

Q-learning is a RL-method where the agent learns about its unknown environment (i.e. the MDP is unknown) through exploration. In each time step *t* the agent chooses an action *a* based on the current state *s*, observes the reward *r* and the next state *s'*, and repeats the process in the new state. Q-learning is then a method that allows the agent to act optimally. Here we will focus on the simplest form of Q-learning algorithms, which can be applied when all states are known to the agent, and the state and action spaces are reasonably small. This simple algorithm uses a table of Q-values for each $(s,a)$ pair, which is then updated in each time step using the update rule in step $k+1$

$$Q_{k+1}(s,a) = Q_k(s,a) + \alpha \left( r(s,a) + \gamma \max \{Q_k(s',a')\} - Q_k(s,a) \right) $$ 

where $\gamma$ is the discount factor as before, and $\alpha$ is a pre-set learning rate. It can be shown that this algorithm converges to the optimal policy of the underlying MDP for certain values of $\alpha$ as long as there  is sufficient exploration. For our case, we set a constant $\alpha=0.1$.

#### OpenAI Gym

We shall use already available simulators for different environments (worlds) using the popular [OpenAI Gym library](https://www.gymlibrary.dev/). It just implements different types of simulators including ATARI games. Although here we will only focus on simple ones, such as the **Chain enviroment** illustrated below.
![alt text](https://chalmersuniversity.box.com/shared/static/6tthbzhpofq9gzlowhr3w8if0xvyxb2b.jpg)
The figure corresponds to an MDP with 5 states $S = \{1,2,3,4,5\}$ and two possible actions $A=\{a,b\}$ in each state. The arrows indicate the resulting transitions for each state-action pair, and the numbers correspond to the rewards for each transition.

## Question 3 (optional)
You are to first familiarize with the framework of [the OpenAI environments](https://www.gymlibrary.dev/), and then implement the Q-learning algorithm for the <code>NChain-v0</code> enviroment depicted above, using default parameters and a learning rate of $\gamma=0.95$. Report the final $Q^*$ table after convergence of the algorithm. For an example on how to do this, you can refer to the Q-learning of the **Frozen lake environment** (<code>q_learning_frozen_lake.ipynb</code>), uploaded on Canvas. Hint: start with a small learning rate.

Note that the NChain environment is not available among the standard environments, you need to load the <code>gym_toytext</code> package, in addition to the standard gym:

<code>
!pip install gym-legacy-toytext<br>
import gym<br>
import gym_toytext<br>
env = gym.make("NChain-v0")<br>
</code>

## Question 4

**4a)** What is the importance of exploration in reinforcement learning? Explain with an example.

**4b)** Explain what makes reinforcement learning different from supervised learning tasks such as regression or classification. 


**4a)**
Exploration is an essentiel part of reinforcement learning, as it enables the agent to learn from its environment and discover new strategies. Without exploration, the agent would continue to follow the same strategy, even if another strategy is more effective. 

Exploration means "trying new actions", actions that the agent haven't tried before. An example of the importance of exploration in reinforcement learning is in the context of training a self-driving car. In this example, the agent (car), needs to learn decisions to manuever safely in traffic and follow traffic rules. To enable self-driving cars the car needs to incorporate exploration in its training. This can be done by  driving in enviroments the car haven't tried before. For example driving in city or on highways, or driving in different speeds. By trying new actions, the self-driving the car can better adopt to new enviroments and take better actions.

**4b)**
Reinforcement learning and supervised learning are different in several ways. 

Foremost, in supervised learning, the model is trained using input data with corresponding output data (labels). The input data and its labels are given to the algorithm to train the model so that it learns a mapping between the two variables. The goal of supervised learning is for the model to correctly classify new, unseen, input data to its corresponding output data (label). 

On the other hand, in reinforcement learning, the algorithm learns to make decisions by interacting and exploring an environment. This is done without explicitly being told what actions to take since the agent receives feedback as rewards based on its actions, so it can learn what action is optimal. The goal of reinforcement learning is to learn a policy, which says which action the agent should take in each state, that maximizes the long-term cumulative reward. 

To summarize, reinforcement learning differs from supervised learning that the model is trained with training data so that the model already has correct answers. In reinforcement learning, there is no answer, and the agent has to learn what actions are most optimal in every state. The model is therefore bound to learn by experience since there isn't any training data. 

## Question 5

**5a)** Give a summary of how a decision tree works and how it extends to random forests.

**5b)** State at least one advantage and one drawback with using random forests over decision trees.

**5a)** A decision tree is a widely used tool when evaluating different actions affected by previously known states/probabilities/actions. The tree splits data into different categories. Consider the choice of dinner. The first categories are, hungry not hungry. If hungry, the next categories are, cooking at home, ordering in or going to a restaurant. If the choice is ordered, the next choice categories are pizza, sushi or Indian. The structure of a decision tree can obviously be used in much more complex situations. When calculating conditional probability, a decision tree is very useful. Dice rolls are a commonly used way to visualise each step in the tree.

A decision tree is easily understandable and effective with more simple problems. The model is not as useable with more complex and multi-dimensional problems, where there are more variables to consider and they can change with different actions and where regression is a possibility. 

Hence the model of random forest. This is a supervised machine-learning algorithm that can handle continuous variables. The basic difference from a decision tree is that it does not rely on a singular decision. It assembles randomized decisions based on several decisions and makes the final decision based on the majority. It does not search for the best prediction. Instead, it makes multiple random predictions. Thus, more diversity is attached, and prediction becomes much smoother.

**5b)** One of the big advantages of using a random forest over a decision tree is the ability to evaluate a complete chain of actions, instead of every action isolated. A random forest can therefore be a useful tool to use over a longer decision-making plan, where different actions have different consequences and where variables can vary over time. 

One drawback of using a random forest over a decision tree is the complexity of the model and the need for programming and computers. A decision tree is easily visualised and can even be shown with just a pen and paper, provided that the problem is relatively small. 


# References
Primer/text based on the following references:
* http://www.cse.chalmers.se/~chrdimi/downloads/book.pdf
* https://github.com/olethrosdc/ml-society-science/blob/master/notes.pdf